In [7]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz

In [10]:
!tar xf spark-3.0.1-bin-hadoop2.7.tgz

In [11]:
!pip install -q findspark

In [12]:
!pip install -q pyspark

In [13]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

In [16]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
spark #verificar se há o objeto spark 

In [18]:
df = spark.read.csv("/content/drive/My Drive/dados/GO AMB/2019_det_amb.csv", header=True, sep=",")


In [19]:
from pyspark.sql import functions as F
from pyspark.sql.types import FloatType


In [20]:
def to_value(v):
    try:
        return float(v.replace(",","."))
    except ValueError:
        return 0.0
udf_to_value = F.udf(to_value, FloatType())

In [21]:
dfmod= df.withColumn('VL_PROCEDIMENTO', udf_to_value(df["VL_PROCEDIMENTO"]))\
   .withColumn('QT_PROCEDIMENTO', udf_to_value(df["QT_PROCEDIMENTO"]))

In [22]:
dfmod.select('CD_TUSS_PROCEDIMENTO', 'VL_PROCEDIMENTO').limi t(10).toPandas()

,CD_TUSS_PROCEDIMENTO,VL_PROCEDIMENTO
0,029,2.670000
1,031,37.619999
2,10101039,95.000000
3,40304361,7.800000
4,004,25.280001
5,029,4.530000
6,030,39.049999
7,031,30.000000
8,029,20.940001
9,031,223.979996


In [ ]:
dfmod.groupBy("CD_TUSS_PROCEDIMENTO")\
    .agg(F.sum("VL_PROCEDIMENTO").alias("Total"))\
    .orderBy("VL_PROCEDIMENTO", ascending=False)\
    .toPandas()

In [26]:
dfmod.printSchema()

root
 |-- ID_EVENTO: string (nullable = true)
 |-- CD_TUSS_PROCEDIMENTO: string (nullable = true)
 |-- QT_PROCEDIMENTO: float (nullable = true)
 |-- VL_PROCEDIMENTO: float (nullable = true)
 |-- VL_PAGO_FORNECEDOR: string (nullable = true)
 |-- CD_TABELA_REFERENCIA: string (nullable = true)
 |-- LG_PACOTE: string (nullable = true)
 |-- IND_TABELA_PROPRIA: string (nullable = true)
 |-- DT_REALIZACAO: string (nullable = true)
 |-- UF_PRESTADOR: string (nullable = true)
 |-- LG_OUTLIER: string (nullable = true)

